Author: Xiang Zhao magelead@outlook.com

TODO:
1. Qbsolv https://github.com/dwavesystems/qbsolv/ ✅
2. Include linear coefficients in diagonal of QUBO quadratic matrix ✅
3. determine penalty factors
4. consider other constraints

In [ ]:
!pip uninstall -q imgaug -y

In [ ]:
!pip install dwave-qbsolv

In [ ]:
!pip install -q qiskit 

In [ ]:
!pip install -q qiskit-optimization

# Airbus challenge

[1] https://arxiv.org/pdf/2102.09621.pdf

[2] https://github.com/magelead/QGSS-2021/blob/main/Lab%202/lab-2.ipynb

# Part I 
Extract QUBO: qubo_matrix(quadratic); qubo_vector(linear); qubo_constant(constant)

In [ ]:
import sympy as sym
import numpy as np

In [ ]:
# Settings for experiments on D-Wave 2000Q, section V.C. of [1]
# This can only be solved by Qbsolv due to large problem size
n=6
m=[2134, 3455, 1866, 1699, 3500, 3332] # TABLE VII of [1], container type 1, unit is kg
N=4
W=8000 # unit is kg

K=24 # number of slack variables: K = n + N + number of bits of binary expansion of W
 
# len(z) equals the number of qubits in QAOA circuit
z=[sym.Symbol(f'z_{i}') for i in range(n*N+K)]
z

[z_0,
 z_1,
 z_2,
 z_3,
 z_4,
 z_5,
 z_6,
 z_7,
 z_8,
 z_9,
 z_10,
 z_11,
 z_12,
 z_13,
 z_14,
 z_15,
 z_16,
 z_17,
 z_18,
 z_19,
 z_20,
 z_21,
 z_22,
 z_23,
 z_24,
 z_25,
 z_26,
 z_27,
 z_28,
 z_29,
 z_30,
 z_31,
 z_32,
 z_33,
 z_34,
 z_35,
 z_36,
 z_37,
 z_38,
 z_39,
 z_40,
 z_41,
 z_42,
 z_43,
 z_44,
 z_45,
 z_46,
 z_47]

In [ ]:
# Settings for a toy experiment

n=2
m=[3, 6] # unit is 100kg, container type 1
N=1
W=4 # unit is 100kg

K=2+1+3 # number of slack variables: K = n + N + number of bits of binary expansion of W
 
    
z=[sym.Symbol(f'z_{i}') for i in range(n*N+K)]
z

[z_0, z_1, z_2, z_3, z_4, z_5, z_6, z_7]

In [ ]:
def f_obj():
    """ objective fucntion """
    exp = 0 # init a expression
    for i in range(n):
        for j in range(N):
            exp = exp - m[i] * z[i*N+j]
    return exp
            

In [ ]:
f_obj()

-2134*z_0 - 2134*z_1 - 1866*z_10 - 1866*z_11 - 1699*z_12 - 1699*z_13 - 1699*z_14 - 1699*z_15 - 3500*z_16 - 3500*z_17 - 3500*z_18 - 3500*z_19 - 2134*z_2 - 3332*z_20 - 3332*z_21 - 3332*z_22 - 3332*z_23 - 2134*z_3 - 3455*z_4 - 3455*z_5 - 3455*z_6 - 3455*z_7 - 1866*z_8 - 1866*z_9

In [ ]:
def f_O_j(j):
    """ no-overlap constriant """
    exp = 0
    for i in range(n):
        exp = exp + z[i*N+j]
    exp = (exp + z[n*N+j] - 1)**2
    
    return exp

In [ ]:
f_O_j(0)

(z_0 + z_12 + z_16 + z_20 + z_24 + z_4 + z_8 - 1)**2

In [ ]:
f_O_j(1)

(z_1 + z_13 + z_17 + z_21 + z_25 + z_5 + z_9 - 1)**2

In [ ]:
def f_O():
    exp = 0
    for j in range(N):
        exp = exp + f_O_j(j)
    return sym.simplify(sym.expand(exp))

In [ ]:
f_O()

z_0**2 + 2*z_0*z_12 + 2*z_0*z_16 + 2*z_0*z_20 + 2*z_0*z_24 + 2*z_0*z_4 + 2*z_0*z_8 - 2*z_0 + z_1**2 + 2*z_1*z_13 + 2*z_1*z_17 + 2*z_1*z_21 + 2*z_1*z_25 + 2*z_1*z_5 + 2*z_1*z_9 - 2*z_1 + z_10**2 + 2*z_10*z_14 + 2*z_10*z_18 + 2*z_10*z_2 + 2*z_10*z_22 + 2*z_10*z_26 + 2*z_10*z_6 - 2*z_10 + z_11**2 + 2*z_11*z_15 + 2*z_11*z_19 + 2*z_11*z_23 + 2*z_11*z_27 + 2*z_11*z_3 + 2*z_11*z_7 - 2*z_11 + z_12**2 + 2*z_12*z_16 + 2*z_12*z_20 + 2*z_12*z_24 + 2*z_12*z_4 + 2*z_12*z_8 - 2*z_12 + z_13**2 + 2*z_13*z_17 + 2*z_13*z_21 + 2*z_13*z_25 + 2*z_13*z_5 + 2*z_13*z_9 - 2*z_13 + z_14**2 + 2*z_14*z_18 + 2*z_14*z_2 + 2*z_14*z_22 + 2*z_14*z_26 + 2*z_14*z_6 - 2*z_14 + z_15**2 + 2*z_15*z_19 + 2*z_15*z_23 + 2*z_15*z_27 + 2*z_15*z_3 + 2*z_15*z_7 - 2*z_15 + z_16**2 + 2*z_16*z_20 + 2*z_16*z_24 + 2*z_16*z_4 + 2*z_16*z_8 - 2*z_16 + z_17**2 + 2*z_17*z_21 + 2*z_17*z_25 + 2*z_17*z_5 + 2*z_17*z_9 - 2*z_17 + z_18**2 + 2*z_18*z_2 + 2*z_18*z_22 + 2*z_18*z_26 + 2*z_18*z_6 - 2*z_18 + z_19**2 + 2*z_19*z_23 + 2*z_19*z_27 + 2*z_19*

In [ ]:
def f_D_i(i):
    """ no-duplicates constriant """
    exp = 0
    for j in range(N):
        exp = exp + z[i*N+j]
    exp = (exp + z[n*N+N+i] - 1)**2
    
    return exp

In [ ]:
f_D_i(0)

(z_0 + z_1 + z_2 + z_28 + z_3 - 1)**2

In [ ]:
f_D_i(1)

(z_29 + z_4 + z_5 + z_6 + z_7 - 1)**2

In [ ]:
def f_D():
    exp = 0
    for i in range(n):
        exp = exp + f_D_i(i)
    return sym.simplify(sym.expand(exp))

In [ ]:
f_D()

z_0**2 + 2*z_0*z_1 + 2*z_0*z_2 + 2*z_0*z_28 + 2*z_0*z_3 - 2*z_0 + z_1**2 + 2*z_1*z_2 + 2*z_1*z_28 + 2*z_1*z_3 - 2*z_1 + z_10**2 + 2*z_10*z_11 + 2*z_10*z_30 + 2*z_10*z_8 + 2*z_10*z_9 - 2*z_10 + z_11**2 + 2*z_11*z_30 + 2*z_11*z_8 + 2*z_11*z_9 - 2*z_11 + z_12**2 + 2*z_12*z_13 + 2*z_12*z_14 + 2*z_12*z_15 + 2*z_12*z_31 - 2*z_12 + z_13**2 + 2*z_13*z_14 + 2*z_13*z_15 + 2*z_13*z_31 - 2*z_13 + z_14**2 + 2*z_14*z_15 + 2*z_14*z_31 - 2*z_14 + z_15**2 + 2*z_15*z_31 - 2*z_15 + z_16**2 + 2*z_16*z_17 + 2*z_16*z_18 + 2*z_16*z_19 + 2*z_16*z_32 - 2*z_16 + z_17**2 + 2*z_17*z_18 + 2*z_17*z_19 + 2*z_17*z_32 - 2*z_17 + z_18**2 + 2*z_18*z_19 + 2*z_18*z_32 - 2*z_18 + z_19**2 + 2*z_19*z_32 - 2*z_19 + z_2**2 + 2*z_2*z_28 + 2*z_2*z_3 - 2*z_2 + z_20**2 + 2*z_20*z_21 + 2*z_20*z_22 + 2*z_20*z_23 + 2*z_20*z_33 - 2*z_20 + z_21**2 + 2*z_21*z_22 + 2*z_21*z_23 + 2*z_21*z_33 - 2*z_21 + z_22**2 + 2*z_22*z_23 + 2*z_22*z_33 - 2*z_22 + z_23**2 + 2*z_23*z_33 - 2*z_23 + z_28**2 + 2*z_28*z_3 - 2*z_28 + z_29**2 + 2*z_29*z_4 + 2*z

In [ ]:
def f_W():
    exp = 0
    for i in range(n):
        for j in range(N):
            exp = exp + m[i] * z[i*N+j]
    for k in range(K-n-N):
        exp = exp + 2**k * z[n*N+n+N+k]
        
    exp = (exp - W)**2
    
    return sym.simplify(sym.expand(exp))
        

In [ ]:
f_W()

4553956*z_0**2 + 9107912*z_0*z_1 + 7964088*z_0*z_10 + 7964088*z_0*z_11 + 7251332*z_0*z_12 + 7251332*z_0*z_13 + 7251332*z_0*z_14 + 7251332*z_0*z_15 + 14938000*z_0*z_16 + 14938000*z_0*z_17 + 14938000*z_0*z_18 + 14938000*z_0*z_19 + 9107912*z_0*z_2 + 14220976*z_0*z_20 + 14220976*z_0*z_21 + 14220976*z_0*z_22 + 14220976*z_0*z_23 + 9107912*z_0*z_3 + 4268*z_0*z_34 + 8536*z_0*z_35 + 17072*z_0*z_36 + 34144*z_0*z_37 + 68288*z_0*z_38 + 136576*z_0*z_39 + 14745940*z_0*z_4 + 273152*z_0*z_40 + 546304*z_0*z_41 + 1092608*z_0*z_42 + 2185216*z_0*z_43 + 4370432*z_0*z_44 + 8740864*z_0*z_45 + 17481728*z_0*z_46 + 34963456*z_0*z_47 + 14745940*z_0*z_5 + 14745940*z_0*z_6 + 14745940*z_0*z_7 + 7964088*z_0*z_8 + 7964088*z_0*z_9 - 34144000*z_0 + 4553956*z_1**2 + 7964088*z_1*z_10 + 7964088*z_1*z_11 + 7251332*z_1*z_12 + 7251332*z_1*z_13 + 7251332*z_1*z_14 + 7251332*z_1*z_15 + 14938000*z_1*z_16 + 14938000*z_1*z_17 + 14938000*z_1*z_18 + 14938000*z_1*z_19 + 9107912*z_1*z_2 + 14220976*z_1*z_20 + 14220976*z_1*z_21 + 142209

In [ ]:
def f_tot(P_W, P_D, P_O):
    exp = f_obj() + P_W*f_W() + P_D*f_D() + P_O*f_O()
    return sym.simplify(sym.expand(exp))
    
    

In [ ]:
qubo = f_tot(1, 1, 1)
qubo

4553958*z_0**2 + 9107914*z_0*z_1 + 7964088*z_0*z_10 + 7964088*z_0*z_11 + 7251334*z_0*z_12 + 7251332*z_0*z_13 + 7251332*z_0*z_14 + 7251332*z_0*z_15 + 14938002*z_0*z_16 + 14938000*z_0*z_17 + 14938000*z_0*z_18 + 14938000*z_0*z_19 + 9107914*z_0*z_2 + 14220978*z_0*z_20 + 14220976*z_0*z_21 + 14220976*z_0*z_22 + 14220976*z_0*z_23 + 2*z_0*z_24 + 2*z_0*z_28 + 9107914*z_0*z_3 + 4268*z_0*z_34 + 8536*z_0*z_35 + 17072*z_0*z_36 + 34144*z_0*z_37 + 68288*z_0*z_38 + 136576*z_0*z_39 + 14745942*z_0*z_4 + 273152*z_0*z_40 + 546304*z_0*z_41 + 1092608*z_0*z_42 + 2185216*z_0*z_43 + 4370432*z_0*z_44 + 8740864*z_0*z_45 + 17481728*z_0*z_46 + 34963456*z_0*z_47 + 14745940*z_0*z_5 + 14745940*z_0*z_6 + 14745940*z_0*z_7 + 7964090*z_0*z_8 + 7964088*z_0*z_9 - 34146138*z_0 + 4553958*z_1**2 + 7964088*z_1*z_10 + 7964088*z_1*z_11 + 7251332*z_1*z_12 + 7251334*z_1*z_13 + 7251332*z_1*z_14 + 7251332*z_1*z_15 + 14938000*z_1*z_16 + 14938002*z_1*z_17 + 14938000*z_1*z_18 + 14938000*z_1*z_19 + 9107914*z_1*z_2 + 14220976*z_1*z_20 + 

In [ ]:
def get_constant(exp):
    subs = [(z[i], 0) for i in range(len(z))]
    constant = exp.subs(subs)
    return int(constant)


In [ ]:
get_constant(qubo)

64000010

In [ ]:
def get_qubo_coefficients(qubo, tolist=True):
    size = len(z)
    quadratic = np.zeros((size, size), dtype=np.int64)
    linear = np.zeros(size, dtype=np.int64)
    constant = get_constant(qubo)
    
    for i in range(size):
        for j in range(size):
            if j >= i:
                quadratic[i, j] =  int(qubo.coeff(z[i]*z[j]))

    for i in range(size):
        linear[i] = int(get_constant(qubo.coeff(z[i])))

    if tolist:
        return quadratic.tolist(), linear.tolist(), constant
    else:
        return quadratic, linear, constant

**Result:**

In [ ]:
quadratic, linear, constant = get_qubo_coefficients(qubo)

In [ ]:
quadratic

[[4553958,
  9107914,
  9107914,
  9107914,
  14745942,
  14745940,
  14745940,
  14745940,
  7964090,
  7964088,
  7964088,
  7964088,
  7251334,
  7251332,
  7251332,
  7251332,
  14938002,
  14938000,
  14938000,
  14938000,
  14220978,
  14220976,
  14220976,
  14220976,
  2,
  0,
  0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  4268,
  8536,
  17072,
  34144,
  68288,
  136576,
  273152,
  546304,
  1092608,
  2185216,
  4370432,
  8740864,
  17481728,
  34963456],
 [0,
  4553958,
  9107914,
  9107914,
  14745940,
  14745942,
  14745940,
  14745940,
  7964088,
  7964090,
  7964088,
  7964088,
  7251332,
  7251334,
  7251332,
  7251332,
  14938000,
  14938002,
  14938000,
  14938000,
  14220976,
  14220978,
  14220976,
  14220976,
  0,
  2,
  0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  4268,
  8536,
  17072,
  34144,
  68288,
  136576,
  273152,
  546304,
  1092608,
  2185216,
  4370432,
  8740864,
  17481728,
  34963456],
 [0,
  0,
  4553958,
  9107914,
  14745940,
  14745940,
  14745942,
  

In [ ]:
linear

[-34146138,
 -34146138,
 -34146138,
 -34146138,
 -55283459,
 -55283459,
 -55283459,
 -55283459,
 -29857870,
 -29857870,
 -29857870,
 -29857870,
 -27185703,
 -27185703,
 -27185703,
 -27185703,
 -56003504,
 -56003504,
 -56003504,
 -56003504,
 -53315336,
 -53315336,
 -53315336,
 -53315336,
 -2,
 -2,
 -2,
 -2,
 -2,
 -2,
 -2,
 -2,
 -2,
 -2,
 -16000,
 -32000,
 -64000,
 -128000,
 -256000,
 -512000,
 -1024000,
 -2048000,
 -4096000,
 -8192000,
 -16384000,
 -32768000,
 -65536000,
 -131072000]

In [ ]:
constant

64000010

**Check**

In [ ]:
def reconstruct_qubo(quadratic, linear, constant):
    size = len(z)
    exp = constant
    for i in range(size):
        for j in range(size):
            exp += quadratic[i, j]*z[i]*z[j]
    
    for i in range(size):
        exp += linear[i]*z[i]

    return exp 

In [ ]:
quadratic, linear, constant = get_qubo_coefficients(qubo, False)
qubo2=reconstruct_qubo(quadratic, linear, constant)

In [ ]:
qubo2

4553958*z_0**2 + 9107914*z_0*z_1 + 7964088*z_0*z_10 + 7964088*z_0*z_11 + 7251334*z_0*z_12 + 7251332*z_0*z_13 + 7251332*z_0*z_14 + 7251332*z_0*z_15 + 14938002*z_0*z_16 + 14938000*z_0*z_17 + 14938000*z_0*z_18 + 14938000*z_0*z_19 + 9107914*z_0*z_2 + 14220978*z_0*z_20 + 14220976*z_0*z_21 + 14220976*z_0*z_22 + 14220976*z_0*z_23 + 2*z_0*z_24 + 2*z_0*z_28 + 9107914*z_0*z_3 + 4268*z_0*z_34 + 8536*z_0*z_35 + 17072*z_0*z_36 + 34144*z_0*z_37 + 68288*z_0*z_38 + 136576*z_0*z_39 + 14745942*z_0*z_4 + 273152*z_0*z_40 + 546304*z_0*z_41 + 1092608*z_0*z_42 + 2185216*z_0*z_43 + 4370432*z_0*z_44 + 8740864*z_0*z_45 + 17481728*z_0*z_46 + 34963456*z_0*z_47 + 14745940*z_0*z_5 + 14745940*z_0*z_6 + 14745940*z_0*z_7 + 7964090*z_0*z_8 + 7964088*z_0*z_9 - 34146138*z_0 + 4553958*z_1**2 + 7964088*z_1*z_10 + 7964088*z_1*z_11 + 7251332*z_1*z_12 + 7251334*z_1*z_13 + 7251332*z_1*z_14 + 7251332*z_1*z_15 + 14938000*z_1*z_16 + 14938002*z_1*z_17 + 14938000*z_1*z_18 + 14938000*z_1*z_19 + 9107914*z_1*z_2 + 14220976*z_1*z_20 + 

In [ ]:
sym.simplify(qubo-qubo2)

0

# Part II



### The QAOA and MinimumEigenOptimizer class
Qiskit provides its own implementation of QAOA which allows us to solve optimization problems with only a few lines of code. <br>
The [`QAOA`](https://qiskit.org/documentation/stubs/qiskit.algorithms.QAOA.html) class in Qiskit is located in `qiskit.algorithms` and directly inherits from the Variational Quantum Eigensolver class [`VQE`](https://qiskit.org/documentation/stubs/qiskit.aqua.algorithms.VQE.html). The initializer for `QAOA` takes the following input parameters, amongst others
- `optimizer`: This argument refers to the classical optimizer used for updating the circuit parameters. Qiskit offers a number of different [optimizers](https://qiskit.org/documentation/apidoc/qiskit.aqua.components.optimizers.html) and you can also define your own by subclassing Qiskit's native [`Optimizer`](https://qiskit.org/documentation/stubs/qiskit.aqua.components.optimizers.Optimizer.html#qiskit.aqua.components.optimizers.Optimizer) class. Some of the basic optimizers offered by Qiskit are the following: <br>
    - [COBYLA](https://qiskit.org/documentation/stubs/qiskit.aqua.components.optimizers.COBYLA.html#qiskit.aqua.components.optimizers.COBYLA): Constrained Optimization By Linear Approximation optimizer.
    - [SLSQP](https://qiskit.org/documentation/stubs/qiskit.aqua.components.optimizers.SLSQP.html#qiskit.aqua.components.optimizers.SLSQP): Sequential Least SQuares Programming optimizer.
    - [ADAM](https://qiskit.org/documentation/stubs/qiskit.aqua.components.optimizers.ADAM.html#qiskit.aqua.components.optimizers.ADAM):  A gradient-based optimization algorithm that is relies on adaptive estimates of lower-order moments.
- `reps`: The number of layers $p$ in the QAOA variational form, i.e. the depth of the algorithm. For higher values of $p$, QAOA can theoretically obtain better results but the quantum circuit becomes deeper and there are more parameters to optimize.
- `initial_point`: Optional initial parameter values (values for $\beta$ and $\gamma$) to start the optimization with.
- `quantum_instance`: The quantum instance to run the algorithm on. This can be a simulator or a real hardware device.
- `callback`: A callback function that can be used to monitor the optimization process.

Below, you will be able to test the effect of some of these arguments on the optimization procedure of QAOA.

The [`MinimumEigenOptimizer`](https://qiskit.org/documentation/tutorials/optimization/3_minimum_eigen_optimizer.html) object provides a wrapper for the optimization process that handles the conversion from a quadratic program to a qubit operator as well as calling a given [`MinimumEigenSolver`](https://qiskit.org/documentation/stubs/qiskit.aqua.algorithms.MinimumEigensolver.html), such as QAOA to obtain the corresponding optimization result. The initializer takes as input the `MinimumEigenSolver` to use for the optimization, and the optimization is run by calling the `optimize` method with a quadratic program as input. 

In [ ]:
from qiskit_optimization import QuadraticProgram

quadratic_program = QuadraticProgram('airbus')

for i in range(len(z)):
    quadratic_program.binary_var(name = f'z_{i}')

quadratic, linear, constant = get_qubo_coefficients(qubo)

quadratic_program.minimize(quadratic=quadratic, linear=linear, constant=constant)

print(quadratic_program.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: airbus

Minimize
 obj: - 31 z_0 - 58 z_1 - 2 z_2 - 2 z_3 - 2 z_4 - 8 z_5 - 16 z_6 - 32 z_7 + [
      22 z_0^2 + 76 z_0*z_1 + 4 z_0*z_2 + 4 z_0*z_3 + 12 z_0*z_5 + 24 z_0*z_6
      + 48 z_0*z_7 + 76 z_1^2 + 4 z_1*z_2 + 4 z_1*z_4 + 24 z_1*z_5 + 48 z_1*z_6
      + 96 z_1*z_7 + 2 z_2^2 + 2 z_3^2 + 2 z_4^2 + 2 z_5^2 + 8 z_5*z_6
      + 16 z_5*z_7 + 8 z_6^2 + 32 z_6*z_7 + 32 z_7^2 ]/2 + 19
Subject To

Bounds
 0 <= z_0 <= 1
 0 <= z_1 <= 1
 0 <= z_2 <= 1
 0 <= z_3 <= 1
 0 <= z_4 <= 1
 0 <= z_5 <= 1
 0 <= z_6 <= 1
 0 <= z_7 <= 1

Binaries
 z_0 z_1 z_2 z_3 z_4 z_5 z_6 z_7
End



In [ ]:
# qiskit.aqua only supports 15 qubits

from qiskit import Aer
from qiskit.algorithms.optimizers import COBYLA, SLSQP, ADAM
from qiskit.algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer

backend = Aer.get_backend('statevector_simulator')
qaoa = QAOA(optimizer = ADAM(), quantum_instance = backend, reps=1, initial_point = [0.1,0.1])
eigen_optimizer = MinimumEigenOptimizer(min_eigen_solver = qaoa)

result = eigen_optimizer.solve(quadratic_program)
print(result)

optimal function value: -3.0
optimal value: [1. 0. 0. 0. 1. 1. 0. 0.]
status: SUCCESS


# Part III

In [ ]:
quadratic, linear, constant = get_qubo_coefficients(qubo, tolist=False)

In [ ]:
linear

array([-31, -58,  -2,  -2,  -2,  -8, -16, -32])

In [ ]:
quadratic

array([[11, 38,  2,  2,  0,  6, 12, 24],
       [ 0, 38,  2,  0,  2, 12, 24, 48],
       [ 0,  0,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  4,  8],
       [ 0,  0,  0,  0,  0,  0,  4, 16],
       [ 0,  0,  0,  0,  0,  0,  0, 16]])

In [ ]:
import copy
def mix_linear_quadratic(linear, quadratic, dict_=True):
    """
    for bianry variable z_i, z_i = z_i^2
    """
    quadratic = copy.deepcopy(quadratic)
    for i in range(len(z)):
        quadratic[i,i] += linear[i]
    
    quadratic_dict = {}
    for i in range(len(z)):
        for j in range(len(z)):
            if j >= i:
                quadratic_dict[(i, j)] = quadratic[i, j]

    if dict_:
        return quadratic_dict
    else:
        return quadratic
   
    

In [ ]:
mix_linear_quadratic(linear, quadratic)

{(0, 0): -20,
 (0, 1): 38,
 (0, 2): 2,
 (0, 3): 2,
 (0, 4): 0,
 (0, 5): 6,
 (0, 6): 12,
 (0, 7): 24,
 (1, 1): -20,
 (1, 2): 2,
 (1, 3): 0,
 (1, 4): 2,
 (1, 5): 12,
 (1, 6): 24,
 (1, 7): 48,
 (2, 2): -1,
 (2, 3): 0,
 (2, 4): 0,
 (2, 5): 0,
 (2, 6): 0,
 (2, 7): 0,
 (3, 3): -1,
 (3, 4): 0,
 (3, 5): 0,
 (3, 6): 0,
 (3, 7): 0,
 (4, 4): -1,
 (4, 5): 0,
 (4, 6): 0,
 (4, 7): 0,
 (5, 5): -7,
 (5, 6): 4,
 (5, 7): 8,
 (6, 6): -12,
 (6, 7): 16,
 (7, 7): -16}

## Qbsolv
A decomposing solver that finds a minimum value of a large quadratic unconstrained binary optimization (QUBO) problem by splitting it into pieces. The pieces are solved using a classical solver running the tabu algorithm. qbsolv also enables configuring a D-Wave system as the solver.

Note: Access to a D-Wave system must be arranged separately.

https://github.com/dwavesystems/qbsolv


https://docs.dwavesys.com/docs/latest/c_gs_3.html#qubo


$z_i=z_i^2 $ for binary variables $z_i$'s



In this case, the proper format of QUBO matrix is upper-triangular with linear terms included in the diagonal. For example:


```python
# [1 2 4
#    2 3
#      1]

from dwave_qbsolv import QBSolv
Q = {(0,0):1, (0,1):2, (0,2):4, (1,1):2, (1,2):3, (2,2):1}
response = QBSolv().sample_qubo(Q)
```



In [ ]:
from dwave_qbsolv import QBSolv

Airbus example

In [ ]:
Q = mix_linear_quadratic(linear, quadratic)
response = QBSolv().sample_qubo(Q)

print("energies=" + str(list(response.data_vectors['energy'])))  # list of optimal function value
print("samples=" + str(list(response.samples()))) # list of optimal value, [{z_0: 0, z_1:0, z_2:1, ...}, ...]

energies=[-64007732.0, -64007510.0, -64007484.0, -64007484.0, -64007465.0, -64007461.0, -64007365.0, -64007362.0, -64007343.0, -64007342.0, -64007342.0, -64007326.0, -64007248.0, -64007239.0, -64007236.0, -64007217.0, -64007196.0, -64007195.0, -64007186.0, -64007186.0]
samples=[{0: 0, 1: 1, 2: 0, 3: 1, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 0, 28: 0, 29: 0, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 0, 36: 1, 37: 0, 38: 1, 39: 0, 40: 0, 41: 0, 42: 1, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0}, {0: 0, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 0, 28: 0, 29: 1, 30: 0, 31: 1, 32: 0, 33: 1, 34: 0, 35: 0, 36: 1, 37: 0, 38: 1, 39: 1, 40: 1, 41: 1, 42: 1, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0}, {0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 1, 10: 

In [ ]:
optimal_value = list(response.samples())[1]  # better try different element in the list, sometimes Qbsolv may give several feasible soultion
replacements = []
for key, val in optimal_value.items():
    replacements.append((z[key], val))

In [ ]:
print('optimal function value:', qubo.subs(replacements))
print('optimal value:', [j for i, j in replacements])
print('status: SUCCESS')

optimal function value: -7500
optimal value: [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
status: SUCCESS


Refert to TABLE IX of [1], the eaxt solution weight is indeed 7500 kg. (n=6, N =4)

Recall that the frst $n\times N$ elemnets of $z$ is $p_{ij}$, i.e. $z[i\times N+j]=p_{ij}$.

 if container $i$ occupies position $j$, $p_{ij}=1$, otherwise $p_{ij}=0$.